In [10]:
import pandas as pd
import numpy as np

In [11]:
nomenclatura_asistencia = {
    2:	'X',
    17:	'A',
    1:	'1TRAB',
    6:	'1DFES',
    5:	'1VAC',
    7:	'1FINJ',
    3:	'B',
    9:	'1INC',
    4:	'C',
    8:	'1FJUS',
    14:	'CS',
    13:	'PGS AP',
    12:	'PGS SA'
}

In [12]:
asistencia = pd.read_csv(r"C:\Users\centro.control\Downloads\IncidenciasPC-01-05-2025-al-31-05-2025.xls", sep='\t')
id_sitios = pd.read_excel(r"C:\Users\centro.control\Downloads\SitioActivosPC-06-04-2025.xlsx")

In [13]:
id_sitios= id_sitios.drop_duplicates(subset=['business_name'], keep='last')

In [14]:
asistencia = asistencia[~ asistencia['incidencia'].isin([17,3])] #quitamos las {bajas: 17, altas:3 }
id_sitios = id_sitios.rename(columns={'business_name': 'sitio'}).drop(columns={'site_manager'})
asistencia['incidencia'] = pd.to_numeric(asistencia['incidencia']).astype('Int64') #convertimos incidencias a entero

In [15]:
#asistencia= pd.merge(id_sitios, asistencia, on='sitio', how='left')
asistencia = pd.merge(asistencia, id_sitios, on='sitio', how='left')

#.drop(columns=['site_manager']) #Left Join de las dos matrices asistencia y id_sitios
#asistencia = asistencia.rename(columns={'key_service': 'Id'}) # renombrar key service: id
asistencia

,id,sitio,nombre_completo,empleo,turno,no_semana,incidencia,fecha,key_service
0,19062,AMERICAN EXPRESS POLANCO,ABREU YAÑEZ JAVIER ELEAZAR,GELB,MATUTINO,19,2,2025-05-01,358
1,13075,CORPORATIVO BENNETTS,ACOSTA JASSO MARIA ALEJANDRA,OPERADOR DE LIMPIEZA,MATUTINO,19,2,2025-05-01,339
2,21530,EUROVI OASIS,ACOSTA MORALES BEATRIZ ANGELICA,OPERADOR DE LIMPIEZA,MATUTINO,19,6,2025-05-01,354
3,21247,LIVERPOOL CIUDAD JARDIN,AGAPITO ANSELMO JUAN MANUEL,OPERADOR DE LIMPIEZA,MATUTINO,19,1,2025-05-01,254
4,3119,ENCUENTRO OCEANIA,AGUEROS GOYCOECHEA RUBEN IVAN,OPERADOR DE LIMPIEZA,MATUTINO,19,1,2025-05-01,256
...,...,...,...,...,...,...,...,...,...
38790,12758,INSTITUTO REGINA,SARABIA LOPEZ PRAXEDIS,OPERADOR DE LIMPIEZA,MIXTO,23,1,2025-05-31,59
38791,13298,EDGEWELL,SEGURA ALVARADO MARIA PATRICIA,SUPERVISOR DE SITIO,MIXTO,23,1,2025-05-31,286
38792,15610,CUMBRES PRIMARIA Y KINDER,TAPIA ROSAS REYNA AYDEE,OPERADOR DE LIMPIEZA,MIXTO,23,1,2025-05-31,80
38793,16418,COLEGIO MONTEVERDE,TREJO MORALES FLORA,OPERADOR DE LIMPIEZA,MIXTO,23,1,2025-05-31,125


In [16]:
def nomenclatura(incidencia):
    return nomenclatura_asistencia.get(incidencia, 'otro')
    #lee la lista y le asigna la clave al valor del diccionario nomenclatura_asistencia
asistencia['Nomenclatura']= list(map(nomenclatura, asistencia['incidencia']))
#se utiliza para aplicar una función dada a cada elemento de un iterable
asistencia.shape

(38795, 10)

In [17]:
asistencia['Significado'] = np.nan   #se rellenan la columna Significado con Nan 
asistencia['Asistencia'] = asistencia['Nomenclatura'].apply(
    lambda x: 1 if x in {'1TRAB','X','RT', 'PGS AP', '1DFES'} else 0
    #para la columna nueva de asistencia se compara si es 1 o es 0 
)
asistencia= asistencia.rename(columns={
    'key_service': 'id', #renombrar columas
    'id': 'candidate_id',
    'no_semana': 'NumWeek'	
})
asistencia.shape
asistencia = asistencia.reindex(columns=[
    'id','candidate_id', 'sitio', 'nombre_completo',
    'empleo','turno','NumWeek','incidencia','fecha','Nomenclatura','Significado','Asistencia'])
asistencia.shape

(38795, 12)